In [1]:
simulation_directory = "/home/novac/Desktop/qv3d-automation/simulation-7.03e14"
file_number = 11

fontsize = 14

In [2]:
import sys
import os
import glob

import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import c, pi, e, m_e, epsilon_0
E0 = 0.5110e6  # [eV] electron rest energy

import sys
sys.path.append('/home/novac/Desktop/qv3d-automation/pyScripts/')
from particles import getParameters, calculateParameters

In [3]:
# Get the parent directory
parent_directory = os.path.abspath(os.path.join(simulation_directory, os.pardir))
em_dotpy = os.path.join(parent_directory, 'pyScripts', 'em.py')

In [4]:
witness_values = !python {em_dotpy} -ol {simulation_directory} {file_number} 2
# witness_headers = !python {em_dotpy} -oH {simulation_directory} {file_number} 2  # unblock to know the variables

driver_values = !python {em_dotpy} -ol {simulation_directory} {file_number} 1
#driver_headers = !python {em_dotpy} -oH {simulation_directory} {file_number} 1  # unblock to know the variables

In [5]:
phase, wmg, stdev_g, W, lamdap, emittance, lamdap=getParameters(witness_values[0])
energy, energySpread, dist, charge = calculateParameters(phase,wmg,stdev_g,W,lamdap)

In [ ]:
# along the acceleration
h5file_directory = os.path.join(simulation_directory, 'h5files')


# Define the pattern to match files starting with 'vs'
pattern = os.path.join(h5file_directory, 'vs*')

# Use glob to find all files matching the pattern
matching_files = glob.glob(pattern)

# Print the matching files
num_phases = 0
for file in matching_files:
    #print(file)
    num_phases = num_phases + 1
print('number of phases (files) are)', num_phases)

In [ ]:
energys = np.array([])
energySpreads = np.array([])
dists = np.array([])
charges = np.array([])

for i in range(num_phases):
    file_number = i + 1
    print('file number is ', file_number, end="\r")
    witness_values = !python {em_dotpy} -ol {simulation_directory} {file_number} 2
    driver_values = !python {em_dotpy} -ol {simulation_directory} {file_number} 1
    phase,wmg,stdev_g,W,lamdap,emittance,lamdap=getParameters(witness_values[0])
    energy, energySpread, dist, charge = calculateParameters(phase,wmg,stdev_g,W,lamdap)
    energys  = np.append(energys, energy)
    energySpreads = np.append(energySpreads, energySpread)
    dists = np.append(dists, dist)
    charges = np.append(charges, charge)

### Plot energy

In [ ]:
fontsize = 14
fig, ax = plt.subplots(figsize=(7,4))
fig.tight_layout(pad=4.)
ax.plot(dists, energys/1e9, c='r', label='witness')
#ax.plot(driverDist*100, driverEnergy/1e6, c='b', label='driver')
    #ax.set_xlim(0,25)
    #ax.set_ylim(0, 500)
ax.set_xlabel('x [cm]', fontsize=fontsize)
ax.set_ylabel('Energy [GeV]', fontsize=fontsize)
    #ax.set_xticks(range(0, 26, 5))
    #ax.set_yticks(range(0, 501, 100))
plt.legend(labels=['witness', 'driver'], loc='lower left')
ax.grid(True, linestyle='--', color='gray', alpha=0.5)
plt.savefig("particles-singleFile-energy.pdf", dpi=100, bbox_inches='tight')
plt.savefig("particles-singleFile-energy.png", dpi=100, bbox_inches='tight')